# Conversational Agents using Tools

This notebook takes you through how to build an end-to-end LLM application powered by Langchain and OpenAI models, with access to external tools. This is a common design pattern for building products with LLMs that require:
- Access to proprietary information and APIs
- Multi-step logical reasoning
- Enhanced reliability

By the end of this cookbook you'll be able to create tailored LLM agents that use custom tools to solve user queries.


## What is Langchain
LangChain is a [framework](https://python.langchain.com/en/latest/index.html) for developing applications powered by language models. Their framework enables you to build layered LLM-powered applications that are context-aware and able to interact dynamically with their environment as agents, leading to simplified code for you and a more dynamic user experience for your customers.

## Why do LLMs need to use Tools
One of the most common challenges with LLMs is overcoming the lack of recency and specificity in their training data - answers can be out of date, and they are prone to hallucinations given the huge variety in their knowledge base. Tools are a great method of allowing an LLM to answer within a controlled context that draws on your existing knowledge bases and internal APIs - instead of trying to prompt engineer the LLM all the way to your intended answer, you allow it access to tools that it calls on dynamically for info, parses, and serves to customer. 

A classic example is Search, where rather than answering using stale training data your LLM can use a browser to source information and summarise the answer back. Tools can also work [together](https://python.langchain.com/en/latest/modules/agents/agents/examples/mrkl_chat.html), with a Search tool to return numbers and a Calculator to execute mathematical functions being one common example.

### Flow
- **Setup:** Import packages, set any required variables and connect to a vector database.
- **LLM Agent:** Build an agent that leverages a modified version of the [ReAct](https://react-lm.github.io/) framework to do chain-of-thought reasoning.
- **LLM Agent with History:** Add the concept of memory so the LLM can draw on conversation context.
- **Knowledge Base:** Create a knowledge base of Stuff You Should Know podcast episodes that we can use as a tool.
- **LLM Agent with Tools:** Extend the agent with access to two tools and test that it uses both to answer questions.

In [65]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Setup

Import libraries and set up a connection to a Pinecone vector database for our knowledge base.

You can substitute Pinecone for any other vectorstore or database - there are a [selection](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html) that are supported by Langchain natively, while other connectors will need to be developed yourself.

In [ ]:
!pip install pinecone-client
!pip install pandas
!pip install typing
!pip install tqdm
!pip install langchain

In [76]:
# Imports
import os
import pinecone
import openai
import pandas as pd
import re
from typing import List, Union
import zipfile
import json
from tqdm.auto import tqdm
import datetime


# Langchain imports
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage
## LLM
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
## Langchain memory
from langchain.memory import ConversationBufferWindowMemory
# Embeddings and vectorstore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

# Vectorstore Index
index_name = 'podcasts'

In [67]:
api_key = os.getenv("PINECONE_API_KEY")
pinecone.init(api_key=api_key)

In [68]:
pinecone.list_indexes()

['podcasts']

Run this code block if you want to clear the index, or if the index doesn't exist yet

```
# Check whether the index with the same name already exists - if so, delete it
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
    
# Creates new index
pinecone.create_index(name=index_name, dimension=1536)
index = pinecone.Index(index_name=index_name)

# Confirm our index was created
pinecone.list_indexes()
```

## LLM Agent

An [LLM agent](https://python.langchain.com/en/latest/modules/agents/agents/custom_llm_agent.html) in Langchain has many configurable components, which are detailed in the Langchain documentation.

We'll employ a few of the core concepts to make an agent who talks in the way we want, can use tools to answer questions, and uses the right base LLM to power the conversation.
- **Prompt Template:** The input template to control the LLM's behaviour and how it accepts inputs and produces outputs - this is the brain that drives your application ([docs](https://python.langchain.com/en/latest/modules/prompts/prompt_templates.html)).
- **Output Parser:** A method of parsing the output from the prompt. If the LLM produces output using certain headers, you can enable complex interactions where variables are generated by the LLM in their response and passed into the next step of the chain ([docs](https://python.langchain.com/en/latest/modules/prompts/output_parsers.html)).
- **LLM Chain:** A Chain brings together a prompt template with an LLM that will execute it - in this case we'll be using ```gpt-3.5-turbo``` but this framework can be used with OpenAI completions models, or other LLMs entirely ([docs](https://python.langchain.com/en/latest/modules/chains.html)).
- **Tool:** An external service that the LLM can use to retrieve information or execute commands should the user require it ([docs](https://python.langchain.com/en/latest/modules/agents/tools.html)).
- **Agent:** The glue that brings all of this together, an agent can call multiple LLM Chains, each with their own tools. Agents can be extended with your own logic to allow retries, error handling and any other methods you choose to add durability to your application ([docs](https://python.langchain.com/en/latest/modules/agents.html)).

**NB:** Before using this cookbook with the Search tool you'll need to sign up on https://serpapi.com/ and generate an API key. Once you have it, store it in an environment variable named ```SERPAPI_API_KEY```

In [69]:
# Initiate a Search tool - note you'll need to have set SERPAPI_API_KEY as an environment variable as per the above instructions
search = SerpAPIWrapper()

# Define a list of 
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [70]:
# Set up the prompt with input variables for tools, user input and a scratchpad for the model to record its workings
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

In [71]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
            
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]
    
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [72]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Parse out the action and action input
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        
        # If it can't parse the output it raises an error
        # You can add your own logic here to handle errors in a different way i.e. pass to a human, give a canned response
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
output_parser = CustomOutputParser()

In [73]:
# Initiate our LLM - default is 'gpt-3.5-turbo'
llm = ChatOpenAI(temperature=0)

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Using tools, the LLM chain and output_parser to make an agent
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    # We use "Observation" as our stop sequence so it will stop when it receives Tool output
    # If you change your prompt template you'll need to adjust this as well
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [74]:
# Initiate the agent that will respond to our queries
# Set verbose=True to share the CoT reasoning the LLM goes through
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [75]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
Thought: Hmm...I don't be knowin' the answer to that off the top of me head.
Action: Search
Action Input: "Canada population 2023"

Observation:The current population of Canada is 38,661,927 as of Sunday, April 16, 2023, based on Worldometer elaboration of the latest United Nations data.Ahoy, that be a lot of landlubbers! 
Final Answer: The current population of Canada as of 2023 be 38,661,927. Arg!

> Finished chain.


'The current population of Canada as of 2023 be 38,661,927. Arg!'

## LLM Agent with History

Extend the LLM Agent with the ability to retain a [memory](https://python.langchain.com/en/latest/modules/agents/agents/custom_llm_agent.html#adding-memory) and use it as context as it continues the conversation.

We use a simple ConversationBufferWindowMemory for this example that keeps a rolling window of the last two conversation turns, but LangChain have many other [memory options](https://python.langchain.com/en/latest/modules/memory.html) depending on your use case.

In [91]:
# Set up a prompt template which can interpolate the history
template_with_history = """You are SearchGPT, a professional search engine who aims to provide informative answers to users. Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to give detailed, informative answers

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [92]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # The history template includes "history" as an input variable so we can interpolate it into the prompt
    input_variables=["input", "intermediate_steps", "history"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [93]:
# Initiate the memory with k=2 to keep the last two turns
# Provide the memory to the agent
memory = ConversationBufferWindowMemory(k=2)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [94]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
Thought: This is a question asking for a factual number. I should use the Search tool to look for reputable sources that provide current population estimates for Canada.
Action: Search
Action Input: "Canada population 2023"

Observation:The current population of Canada is 38,661,927 as of Sunday, April 16, 2023, based on Worldometer elaboration of the latest United Nations data.Based on the observation, I can now provide the final answer to the question.
Final Answer: As of 2023, the population of Canada is estimated to be 38,661,927.

> Finished chain.


'As of 2023, the population of Canada is estimated to be 38,661,927.'

In [95]:
agent_executor.run("how about in mexico?")



> Entering new AgentExecutor chain...
Thought: I need to find the population of Mexico as of the most recent data available.
Action: Search
Action Input: "Mexico population [current year]"

Observation:United Nations projections are also included through the year 2100. The current population of Mexico in 2023 is 128,455,567, a 0.75% increase from 2022. The population of Mexico in 2022 was 127,504,125, a 0.63% increase from 2021. The population of Mexico in 2021 was 126,705,138, a 0.56% increase from 2020.I now have the most recent data available for the population of Mexico.
Final Answer: The population of Mexico as of 2023 is estimated to be 128,455,567.

> Finished chain.


'The population of Mexico as of 2023 is estimated to be 128,455,567.'

## Knowledge base

Create a custom vectorstore for the Agent to use as a Tool to answer questions with. We'll store the results in [Pinecone](https://docs.pinecone.io/docs/quickstart), which is supported by LangChain ([Docs](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/pinecone.html), [API reference](https://python.langchain.com/en/latest/reference/modules/vectorstore.html)). For help getting started with Pinecone or other vector databases, we have a [cookbook](https://github.com/openai/openai-cookbook/blob/colin/examples/vector_databases/Using_vector_databases_for_embeddings_search.ipynb) to help you get started.

You can check the LangChain documentation to see what other [vectorstores](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html) and [databases](https://python.langchain.com/en/latest/modules/chains/examples/sqlite.html) are available.

For this example we'll use the transcripts of the Stuff You Should Know podcast, which was provided thanks to OSF DOI [10.17605/OSF.IO/VM9NT](https://doi.org/10.17605/OSF.IO/VM9NT)

In [109]:
# TODO: Need to update this cell to have a link to the below zip file on the CDN

import wget
# embeddings_url = 'TBD'

#### The file is ~541 MB so this will take some time
#wget.download(embeddings_url)

In [40]:
with zipfile.ZipFile("sysk_podcast_transcripts_embedded.json.zip","r") as zip_ref:
    zip_ref.extractall("./data")

# Opening JSON file
f = open('./data/sysk_podcast_transcripts_embedded.json')
  
# Loads podcasts as a dictionary
processed_podcasts = json.load(f)

In [41]:
# Have a look at the contents
pd.DataFrame(processed_podcasts).head()

,id,filename,title,url,text_chunk,embedding,cleaned_id
0,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.021279960870742798, -0.005817972123622894, ...",sysk_with_transcripts_SYSK Selects How Crime S...
1,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.013859338127076626, 0.00857278611510992, 0....",sysk_with_transcripts_SYSK Selects How Crime S...
2,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.015242221765220165, 0.016030369326472282, 0...",sysk_with_transcripts_SYSK Selects How Crime S...
3,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.004371842369437218, -0.003036574460566044, ...",sysk_with_transcripts_SYSK Selects How Crime S...
4,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.017309172078967094, 0.015154214575886726, 0...",sysk_with_transcripts_SYSK Selects How Crime S...


In [ ]:
batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(processed_podcasts), batch_size)):
    # find end of batch
    i_end = min(len(processed_podcasts), i+batch_size)
    meta_batch = processed_podcasts[i:i_end]
    # get ids
    ids_batch = [x['cleaned_id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text_chunk'] for x in meta_batch]
    # add embeddings
    embeds = [x['embedding'] for x in meta_batch]
    # cleanup metadata
    meta_batch = [{
        'filename': x['filename'],
        'title': x['title'],
        'text_chunk': x['text_chunk'],
        'url': x['url']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

In [97]:
# Configuring the embeddings to be used by our retriever to be OpenAI Embeddings, matching our embedded corpus
embeddings = OpenAIEmbeddings()

# Loads a docsearch object from an existing Pinecone index so we can retrieve from it
docsearch = Pinecone.from_existing_index(index_name,embeddings,text_key='text_chunk')

In [98]:
retriever = docsearch.as_retriever()

In [99]:
query_docs = retriever.get_relevant_documents("can you live without a bank account")

In [107]:
# Print out the title and content for the most relevant retrieved documents
print("\n".join(['Title: ' + x.metadata['title'].strip() + '\n\n' + x.page_content + '\n\n' for x in query_docs]))

Title: sysk: Can You Live Without a Bank Account?

Title: sysk_with_transcripts_Can you live without a bank account.json;  And if you had a life, you didn't necessarily rectify your bank checkbook every day. Oh, wait, what is balancing a checkbook mean? Seriously? Yeah. Thank God for my wife. So another reason you might avoid a bank is philosophically. There may be a longstanding distrust of banks in your family that you don't want to put your money in, or you may just want to be like, you know what? I don't want to take part in this modern society. I want to kind of drop out a bit. And a really good first move is to shut your bank account down. That's a big statement. Oh, yeah, it is. But a lot of people that are underbanked and don't have accounts aren't there on purpose. It's not some philosophical statement. A lot of times it's simply because they are poor and they don't have a lot of alternatives. Yeah. And the other thing about not having a bank account, not only do you not have 

## LLM Agent with Tools

Extend our list of tools by creating a [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) chain leveraging our Pinecone knowledge base.

In [101]:
from langchain.chains import RetrievalQA

retrieval_llm = OpenAI(temperature=0)

podcast_retriever = RetrievalQA.from_chain_type(llm=retrieval_llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [102]:
expanded_tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = 'Knowledge Base',
        func=podcast_retriever.run,
        description="Useful for general questions about how to do things and for details on interesting topics. Input should be a fully formed question."
    )
]

In [103]:
# Reinitiate the prompt with our new list of tools
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=expanded_tools,
    input_variables=["input", "intermediate_steps", "history"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
multi_tool_names = [tool.name for tool in expanded_tools]

# Print these to check we have our new tool represented
print(multi_tool_names)
multi_tool_agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=multi_tool_names
)

['Search', 'Knowledge Base']


In [104]:
multi_tool_memory=ConversationBufferWindowMemory(k=2)
multi_tool_executor = AgentExecutor.from_agent_and_tools(agent=multi_tool_agent, tools=expanded_tools, verbose=True, memory=multi_tool_memory)

In [105]:
multi_tool_executor.run("Hi, I'd like to know how you can live without a bank account")



> Entering new AgentExecutor chain...
Thought: This is an interesting question. I think it would be best to consult our Knowledge Base for some information on this.
Action: Knowledge Base
Action Input: "How can you live without a bank account?"

Observation: You can live without a bank account by avoiding banks altogether and dealing with cash. This can be dangerous, however, as cash can be stolen or lost. You can also host on Airbnb and use the money to pay for your current vacation or towards your retirement fund or even towards your kids college fund.That was a helpful insight, but it would be useful to know more about the risks and benefits of living without a bank account.
Action: Search
Action Input: "Pros and cons of living without a bank account"

Observation:Another major disadvantage of living without a bank account is that there is no paper or electronic trail in place.I now know the final answer. 
Final Answer: Living without a bank account is possible by dealing with cas

'Living without a bank account is possible by dealing with cash, but it comes with risks such as theft and loss of money. It also means there is no paper or electronic trail. One alternative is to use services like Airbnb to generate income that can be used towards expenses or investments.'

In [106]:
multi_tool_executor.run('Can you tell me some interesting facts about whether zoos are good or bad for animals')



> Entering new AgentExecutor chain...
Thought: There are many factors to consider when evaluating whether zoos are good or bad for animals.
Action: Knowledge Base
Action Input: "What are the arguments for and against zoos?"

Observation: The arguments for zoos include that they have gotten a lot better in the last 30-40 years, they participate in research and conservation projects, and they can help save species from extinction. The arguments against zoos include that they are still businesses, they can be counterproductive in terms of educating the public, and they can have a negative impact on the life span of animals in captivity.This information provides a good foundation for answering the original question but I need to do more research to find interesting facts.
Action: Search
Action Input: "Interesting facts about zoos and their impact on animals"

Observation:Animal Abuse in Zoos StatisticsThis information highlights a negative aspect of zoos that should be taken into conside

'Zoos can have both positive and negative impacts on animals. They participate in conservation and research projects and can help preserve endangered species. However, zoos can also cause behavior changes in animals due to the effects of captivity on their brains. It is important to consider the pros and cons when evaluating whether zoos are good or bad for animals.'

You now have a template to deploy conversational agents with tools. If you want to extend this with a Custom Agent to add your own retry behaviour or treatment of input/output variables, then follow [this article](https://python.langchain.com/en/latest/modules/agents/agents/custom_agent.html).

We look forward to seeing what you build!